In [1]:
import torch
import train

import sys
import os.path
import math
import json

import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from tqdm import tqdm

import config
import data
import model
import utils

log = torch.load('logs/2019-05-27_14:24:02.pth')
tokens = len(log['vocab']['question']) + 1

net = torch.nn.DataParallel(model.Net(tokens)).cuda()
net.load_state_dict(log['weights'])

/datasets/home/home-03/99/999/tfarnan/ECE285/DDST-VQA/vqa_pytorch/cyanogenoid_code/model.py:90: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(w)
/datasets/home/home-03/99/999/tfarnan/ECE285/DDST-VQA/vqa_pytorch/cyanogenoid_code/model.py:86: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.embedding.weight)
/datasets/home/home-03/99/999/tfarnan/ECE285/DDST-VQA/vqa_pytorch/cyanogenoid_code/model.py:44: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight)


In [2]:
def run_eval(net, loader, optimizer, tracker, train=False, prefix='', epoch=0):
    """ Run an epoch over the given loader """

    net.eval()
    tracker_class, tracker_params = tracker.MeanMonitor, {}
    answ = []
    idxs = []
    accs = []

    tq = tqdm(loader, desc='{} E{:03d}'.format(prefix, epoch), ncols=0)
    acc_tracker = tracker.track('{}_acc'.format(prefix), tracker_class(**tracker_params))

    log_softmax = nn.LogSoftmax().cuda()
    for v, q, a, idx, q_len in tq:
        var_params = {
            'volatile': not train,
            'requires_grad': False,
        }
        v = Variable(v.cuda(async=True), **var_params)
        q = Variable(q.cuda(async=True), **var_params)
        a = Variable(a.cuda(async=True), **var_params)
        q_len = Variable(q_len.cuda(async=True), **var_params)

        out = net(v, q, q_len)
        acc = utils.batch_accuracy(out.data, a.data).cpu()

        _, answer = out.data.cpu().max(dim=1)
        answ.append(answer.view(-1))
        accs.append(acc.view(-1))
        idxs.append(idx.view(-1).clone())


    if not train:
        answ = list(torch.cat(answ, dim=0))
        accs = list(torch.cat(accs, dim=0))
        idxs = list(torch.cat(idxs, dim=0))
        return answ, accs, idxs

In [3]:
def eval(net):
    val_loader = data.get_loader(val=True)
    optimizer = optim.Adam([p for p in net.parameters() if p.requires_grad])
    
    tracker = utils.Tracker()
    config_as_dict = {k: v for k, v in vars(config).items() if not k.startswith('__')}
    r = run_eval(net, val_loader, optimizer, tracker, train=False, prefix='val', epoch=0)
    return r

In [4]:
y = eval(net)

val E000:   0% 0/6699 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
val E000: 100% 6699/6699 [21:10<00:00,  5.27it/s]


In [5]:
print(len(y[1]))
print(sum(y[1]))
print(max(y[2]))

214354
tensor(115398.5781)
tensor(214353)


In [6]:
total = sum(y[1])/len(y[1])

In [7]:
print(total)

tensor(0.5384)


In [8]:
print(sum(1 for i in y[1] if i >= .5))

32770
